In [1]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf

from gcn.utils import *
from gcn.models import GCN, MLP

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data('citeseer')

In [3]:
features = preprocess_features(features)
support = [preprocess_adj(adj)] # Graph Laplacian from adjacency matrix , in a compressed form through sparse_to_tuple function
num_supports = 1
model_func = GCN

In [4]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

In [5]:
from gcn.layers import *
from gcn.metrics import *

flags = tf.app.flags
FLAGS = flags.FLAGS


class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError
        # define it in the child class as needed 

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build() # first call the child class' build

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)


class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)
        self.learning_rate = 0.01
        self.hidden1 = 16
        self.weight_decay = 5e-4
        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(self.learning_rate)

        self.build() # from Model 

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += self.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=self.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            sparse_inputs=True,
                                            logging=self.logging))

        self.layers.append(GraphConvolution(input_dim=self.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x,
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [6]:
# Create model
model = GCN(placeholders, input_dim=features[2][1], logging=True)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [7]:
# Initialize session
sess = tf.Session()

In [8]:
# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)

In [9]:
# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

In [10]:
# Train model
epochs = 200
dropout = 0.5
early_stopping = 10
for epoch in range(epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']:dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > early_stopping and cost_val[-1] > np.mean(cost_val[-(early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))


Epoch: 0001 train_loss= 1.79988 train_acc= 0.12500 val_loss= 1.79539 val_acc= 0.33400 time= 0.04680
Epoch: 0002 train_loss= 1.79321 train_acc= 0.41667 val_loss= 1.79180 val_acc= 0.47000 time= 0.02956
Epoch: 0003 train_loss= 1.78746 train_acc= 0.55833 val_loss= 1.78886 val_acc= 0.50800 time= 0.02776
Epoch: 0004 train_loss= 1.78179 train_acc= 0.68333 val_loss= 1.78654 val_acc= 0.50800 time= 0.02661
Epoch: 0005 train_loss= 1.77314 train_acc= 0.73333 val_loss= 1.78472 val_acc= 0.51400 time= 0.02699
Epoch: 0006 train_loss= 1.76781 train_acc= 0.80833 val_loss= 1.78308 val_acc= 0.49800 time= 0.02725
Epoch: 0007 train_loss= 1.76107 train_acc= 0.80000 val_loss= 1.78149 val_acc= 0.49600 time= 0.02664
Epoch: 0008 train_loss= 1.75164 train_acc= 0.79167 val_loss= 1.77997 val_acc= 0.50400 time= 0.02853
Epoch: 0009 train_loss= 1.74751 train_acc= 0.77500 val_loss= 1.77841 val_acc= 0.51400 time= 0.03050
Epoch: 0010 train_loss= 1.74177 train_acc= 0.83333 val_loss= 1.77692 val_acc= 0.52000 time= 0.03085


In [11]:
model.layers[0].vars

{'weights_0': <tf.Variable 'gcn/graphconvolution_1_vars/weights_0:0' shape=(3703, 16) dtype=float32_ref>}

In [12]:
model.layers[1].vars.values()

[<tf.Variable 'gcn/graphconvolution_2_vars/weights_0:0' shape=(16, 6) dtype=float32_ref>]

In [15]:
print((sess.run(model.layers[1].vars.values()[0])))

[[ 2.2140489   1.6216251  -1.9067668  -1.8295588  -2.0481884   2.3971107 ]
 [ 1.4534065   1.9322166  -1.465002    2.1159115  -1.8303217  -2.0061352 ]
 [-2.4929097   2.4627924  -2.0897002  -2.0380554   1.8251437   1.9101021 ]
 [ 1.9273038  -2.487797   -2.4137433   1.6289151  -2.4051871   2.0461771 ]
 [-1.8334296  -1.5462577   2.1881366   2.1788633  -1.7635618  -2.11568   ]
 [ 2.0998175   1.4780444  -1.9124211  -2.0544236   2.1584485  -1.972863  ]
 [-2.101468   -1.9553187  -2.1900764   2.1582403   1.6812649   1.7475207 ]
 [-2.2944875   2.4202244  -2.1072235  -1.7602531   2.0247805   1.1951679 ]
 [ 1.0059295   1.7078187  -0.0147922   1.445493   -2.1847682  -2.296212  ]
 [-2.4198601   2.076082    2.0305727   1.669824   -2.2009742  -1.7315618 ]
 [-1.6457802  -1.693406    2.4736502  -2.060135    2.1198807   1.4583666 ]
 [-1.490863   -2.0959952   2.1890795  -1.8859533   1.7571442   1.6943876 ]
 [-0.94960743 -2.0994654   2.3312528  -2.3186522  -2.2903838   1.9181403 ]
 [-1.7107073  -2.082297  

In [ ]:
# Can we do a clustering on the last part? If so, what happens? 